In [7]:
!pip install --upgrade huggingface_hub 

  Obtaining dependency information for huggingface_hub from https://files.pythonhosted.org/packages/69/d6/73f9d1b7c4da5f0544bc17680d0fa9932445423b90cd38e1ee77d001a4f5/huggingface_hub-0.23.4-py3-none-any.whl.metadata
  Using cached huggingface_hub-0.23.4-py3-none-any.whl.metadata (12 kB)
Using cached huggingface_hub-0.23.4-py3-none-any.whl (402 kB)
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.17.3
    Uninstalling huggingface-hub-0.17.3:
      Successfully uninstalled huggingface-hub-0.17.3


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tokenizers 0.14.1 requires huggingface_hub<0.18,>=0.16.4, but you have huggingface-hub 0.23.4 which is incompatible.

[notice] A new release of pip is available: 23.2.1 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
!pip install --upgrade tokenizers

  Obtaining dependency information for tokenizers from https://files.pythonhosted.org/packages/65/8e/6d7d72b28f22c422cff8beae10ac3c2e4376b9be721ef8167b7eecd1da62/tokenizers-0.19.1-cp311-none-win_amd64.whl.metadata
  Using cached tokenizers-0.19.1-cp311-none-win_amd64.whl.metadata (6.9 kB)
Using cached tokenizers-0.19.1-cp311-none-win_amd64.whl (2.2 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.14.1
    Uninstalling tokenizers-0.14.1:
      Successfully uninstalled tokenizers-0.14.1


ERROR: Could not install packages due to an OSError: [WinError 5] Отказано в доступе: 'E:\\PyCharm Community Edition 2021.1.1\\projects\\LLM_LoRa\\venv\\Lib\\site-packages\\~okenizers\\tokenizers.cp311-win_amd64.pyd'
Check the permissions.


[notice] A new release of pip is available: 23.2.1 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
!pip install peft -q
!pip install -i https://pypi.org/simple/ bitsandbytes -q
!pip install accelerate -q
!pip install datasets -q
!pip install zstandard -q
!pip install jsonlines -q
!pip install sentence_transformers -q
!pip install catboost -q


[notice] A new release of pip is available: 23.2.1 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tokenizers 0.14.1 requires huggingface_hub<0.18,>=0.16.4, but you have huggingface-hub 0.23.4 which is incompatible.

[notice] A new release of pip is available: 23.2.1 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.1.1
[notice] To

In [9]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
  Using cached https://download.pytorch.org/whl/cu118/torch-2.3.1%2Bcu118-cp311-cp311-win_amd64.whl (2673.0 MB)
  Attempting uninstall: torch
    Found existing installation: torch 2.2.0
    Uninstalling torch-2.2.0:
      Successfully uninstalled torch-2.2.0


  You can safely remove it manually.

[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
# from sentence_transformers import SentenceTransformer
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
import torch
# from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from tqdm import tqdm
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from tabulate import tabulate
from sklearn.metrics import classification_report, f1_score, roc_auc_score, accuracy_score, recall_score, PrecisionRecallDisplay

In [2]:


MODEL_NAME = "IlyaGusev/saiga_7b_lora"
DEFAULT_MESSAGE_TEMPLATE = "<s>{role}\n{content}</s>\n"
DEFAULT_SYSTEM_PROMPT = "Ты — переводчик. Ты переводишь текст с русского, на текст, как будто он был переведён с китайского. Пример: 'Меня зовут Иван, живу в России и я работаю в шахте. Читал труды китайской партии, и мне понравилось.' -> 'Я простой русский рабочий Иван, работать шахта, жить Россия. Читать книга Китай партия, много нравиться."

class Conversation:
    def __init__(
        self,
        message_template=DEFAULT_MESSAGE_TEMPLATE,
        system_prompt=DEFAULT_SYSTEM_PROMPT,
        start_token_id=1,
        bot_token_id=9225
    ):
        self.message_template = message_template
        self.start_token_id = start_token_id
        self.bot_token_id = bot_token_id
        self.messages = [{
            "role": "system",
            "content": system_prompt
        }]

    def get_start_token_id(self):
        return self.start_token_id

    def get_bot_token_id(self):
        return self.bot_token_id

    def add_user_message(self, message):
        self.messages.append({
            "role": "user",
            "content": message
        })

    def add_bot_message(self, message):
        self.messages.append({
            "role": "bot",
            "content": message
        })

    def get_prompt(self, tokenizer):
        final_text = ""
        for message in self.messages:
            message_text = self.message_template.format(**message)
            final_text += message_text
        final_text += tokenizer.decode([self.start_token_id, self.bot_token_id])
        return final_text.strip()


def generate(model, tokenizer, prompt, generation_config):
    data = tokenizer(prompt, return_tensors="pt")
    data = {k: v.to(model.device) for k, v in data.items()}
    output_ids = model.generate(
        **data,
        generation_config=generation_config
    )[0]
    output_ids = output_ids[len(data["input_ids"][0]):]
    output = tokenizer.decode(output_ids, skip_special_tokens=True)
    return output.strip()



In [3]:
config = PeftConfig.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto"
)


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
print("model loaded")

model loaded


In [5]:
model = PeftModel.from_pretrained(
    model,
    MODEL_NAME,
    torch_dtype=torch.float16,
    is_trainable = True,
)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
print(generation_config)

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message


GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_new_tokens": 1536,
  "no_repeat_ngram_size": 15,
  "pad_token_id": 0,
  "repetition_penalty": 1.1,
  "temperature": 0.2,
  "top_k": 40,
  "top_p": 0.9
}



In [6]:
import pandas as pd

In [7]:
df = pd.read_csv("translate.csv", sep="|")

In [8]:
print(df.iloc[11])

translated    Моя семья счастливый когда партия выдать много...
original            Моя семья счастлива, когда дома много риса.
Name: 11, dtype: object


In [9]:
def get_prompt(question):
    return f"""
    Текст, который нужно перевести: {question}
    Переведи с русского так, как будто этот текст был переведён с китайского в переводчике."""

# def get_promt_lev(summary, question, answer):
#     return f"""Высказывание 1: {summary}
#     Высказывание 2: {question + answer}
#     Сравни Высказывание 1 и Высказывание 2 и скажи правдиво ли второе, написав 1 если правдиво и 0 если наоборот"""

In [10]:

def answer(question, translate):
    inputs = get_prompt(question)

    conversation = Conversation()
    conversation.add_user_message(inputs)
    prompt = conversation.get_prompt(tokenizer)

    output = generate(model, tokenizer, prompt, generation_config)
    print(inputs)
    print(output)
    print(output.lower().split(" "))
    print("Translated: ", translate)
    print()
    print("==============================")
    print()
    return  output

In [11]:
import pandas as pd
import json
import os

def dataset_to_json(dataset, filename):
    json_objects = []
    if not os.path.isfile(filename):
        with open(filename, 'w', encoding="utf-8") as file:
            file.write("")

    for index, row in dataset.iterrows():        
        system_message = "Ты — переводчик. Ты переводишь текст с русского, на текст, как будто он был переведён с китайского. Пример: 'Меня зовут Иван, живу в России и я работаю в шахте. Читал труды китайской партии, и мне понравилось.' -> 'Я простой русский рабочий Иван, работать шахта, жить Россия. Читать книга Китай партия, много нравиться. "
        user_message = f"{row['original']}"
        bot_message = row['translated']

        json_object = {
            "system": system_message,
            "user": user_message,
            "bot": bot_message
        }

        json_objects.append(json_object)
        with open(filename, 'a', encoding='utf-8') as file:
          file.write(json.dumps(json_object, ensure_ascii=False) + "\n")

    return json_objects


In [20]:
train_size = 1
train_dataset=df.sample(frac=train_size,random_state=200)
test_dataset=df.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)


print("FULL Dataset: {}".format(df.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))
dataset_to_json(train_dataset, "train.json")
dataset_to_json(test_dataset, "test.json")


FULL Dataset: (34, 2)
TRAIN Dataset: (34, 2)
TEST Dataset: (0, 2)


[]

In [21]:
from datasets import load_dataset
data = load_dataset(
    "json",
    data_files={
                'train' : 'train.json' ,
                'validation' : 'test.json'
    }
)
data

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['system', 'user', 'bot'],
        num_rows: 34
    })
    validation: Dataset({
        features: ['system', 'user', 'bot'],
        num_rows: 1
    })
})

In [22]:
CUTOFF_LEN = 3584

def generate_prompt(data_point):
    promt = f"""<s>system
{data_point['system']}</s><s>user
{data_point['user']}</s><s>bot
{data_point['bot']}</s>"""
    #     print(promt)
    return promt


def tokenize (prompt, add_eos_token=True):
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=CUTOFF_LEN,
        padding=False,
        return_tensors=None,
    )
    if (
        result["input_ids"][-1] != tokenizer.eos_token_id and len(result["input_ids"]) < CUTOFF_LEN
        and add_eos_token
    ):

        result["input_ids"].append(tokenizer.eos_token_id)
        result["attention_mask"].append(1)



    result["labels"] = result["input_ids"].copy()

    return result

def generate_and_tokenize_prompt(data_point):
    full_prompt = generate_prompt(data_point)
    tokenized_full_prompt = tokenize(full_prompt)
#     print(tokenized_full_prompt)
    return tokenized_full_prompt

In [23]:
train_data = (
    data["train"].map(generate_and_tokenize_prompt)
)

val_data = (
    data["validation"].map(generate_and_tokenize_prompt)
)

Map:   0%|          | 0/34 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [24]:
import torch
import pandas as pd
from peft import PeftModel, PeftConfig
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForTokenClassification, AutoConfig, GenerationConfig
from transformers import Trainer, TrainingArguments, logging, TrainerCallback, TrainerState, TrainerControl, BitsAndBytesConfig
from transformers.trainer_utils import PREFIX_CHECKPOINT_DIR
from peft import get_peft_model, LoraConfig, prepare_model_for_kbit_training
import torch.nn.functional as F
from datasets import load_dataset
import time
from typing import Any, List, Mapping, Optional
import transformers
import os
from pathlib import Path
BATCH_SIZE = 2
MICRO_BATCH_SIZE = 1
GRADIENT_ACCUMULATION_STEPS = BATCH_SIZE // MICRO_BATCH_SIZE
LEARNING_RATE = 3e-4
TRAIN_STEPS = 20
OUTPUT_DIR = "model"

training_arguments = transformers.TrainingArguments(
            per_device_train_batch_size=MICRO_BATCH_SIZE,
            gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
#             warmup_steps=200,
            max_steps=TRAIN_STEPS,
            learning_rate=LEARNING_RATE,
            fp16=True,
            logging_steps=10,
            optim="adamw_torch",
            evaluation_strategy="steps",
            save_strategy="steps",
            eval_steps=10,
            save_steps=10,
            output_dir=OUTPUT_DIR,
            save_total_limit=10,
            load_best_model_at_end=True,
            report_to=None,
            overwrite_output_dir=True, # Overwrite the content of the output dir
)

E:\PyCharm Community Edition 2021.1.1\projects\LLM_LoRa\venv\Lib\site-packages\transformers\training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [25]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [42]:
!pip install accelerate==0.27.2
!pip install torch==2.2.0


[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/198.6 MB ? eta -:--:--
   ---------------------------------------- 0.1/198.6 MB 2.8 MB/s eta 0:01:11
   ---------------------------------------- 0.2/198.6 MB 2.4 MB/s eta 0:01:23
   ---------------------------------------- 0.3/198.6 MB 2.7 MB/s eta 0:01:14
   ---------------------------------------- 0.5/198.6 MB 2.5 MB/s eta 0:01:19
   ---------------------------------------- 0.6/198.6 MB 2.5 MB/s eta 0:01:20
   ---------------------------------------- 0.7/198.6 MB 2.5 MB/s eta 0:01:19
   ---------------------------------------- 0.8/198.6 MB 2.6 MB/s eta 0:01:18
   ---------------------------------------- 0.9/198.6 MB 2.6 MB/s eta 0:01:17
   ---------------------------------------- 1.1/198.6 MB 2.5 MB/s eta 0:01:20
   ---------------------------------------- 1.2/198.6 MB 2.5 MB/s eta 0:01:21
   ---------------------------------------- 1.3/198.6 MB 2.5 MB/s eta 0:01:20
   ---------------------------------------- 1.4/198.6 MB 2.5 MB/s eta 0

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.3.1+cu118 requires torch==2.3.1+cu118, but you have torch 2.2.0 which is incompatible.
torchvision 0.18.1+cu118 requires torch==2.3.1+cu118, but you have torch 2.2.0 which is incompatible.

[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [26]:
from transformers import DataCollatorForSeq2Seq, Trainer, TrainingArguments
import transformers
data_collator = transformers.DataCollatorForSeq2Seq(
    tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True
)

trainer = Trainer(
    model=model,
    args=training_arguments,
    train_dataset=train_data,
    eval_dataset=val_data,
    data_collator=data_collator,
)

model = torch.compile(model)
trainer.train()


max_steps is given, it will override any value given in num_train_epochs
We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)
E:\PyCharm Community Edition 2021.1.1\projects\LLM_LoRa\venv\Lib\site-packages\transformers\models\llama\modeling_llama.py:648: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Step,Training Loss,Validation Loss
10,1.312100,0.395169
20,0.464800,0.279345


TrainOutput(global_step=20, training_loss=0.8884269237518311, metrics={'train_runtime': 32.7561, 'train_samples_per_second': 1.221, 'train_steps_per_second': 0.611, 'total_flos': 291950319697920.0, 'train_loss': 0.8884269237518311, 'epoch': 1.1764705882352942})

In [27]:
df["answer"] =  df.apply(lambda x: answer(x.summary, x.question, x.answer, x.is_hallucination), axis=1)

AttributeError: 'Series' object has no attribute 'summary'

In [28]:
from sklearn import metrics
accuracy = metrics.accuracy_score(df["is_hallucination"],df["answer"])
f1_score_micro = metrics.f1_score(df["is_hallucination"],df["answer"], average='micro')
f1_score_macro = metrics.f1_score(df["is_hallucination"],df["answer"], average='macro')
print(f"Accuracy Score = {accuracy}")
print(f"F1 Score (Micro) = {f1_score_micro}")
print(f"F1 Score (Macro) = {f1_score_macro}")

KeyError: 'is_hallucination'

In [29]:
model.save_pretrained(OUTPUT_DIR)

# Catboost


In [ ]:
df["output"] =  df.apply(lambda x: answer(x.summary, x.question, x.answer, x.is_hallucination), axis=1)

Выходные данные были обрезаны до нескольких последних строк (5000).
    Вопрос: В каком году был убит этот человек?
    Ответ на вопрос: в 1959 году
    Оцени согласованность Ответа на вопрос, Вопроса и Контекста по шкале от 0 до 1, где 0 - полное несоответсвие, а 1 - полное соответсвие.
Ответ на вопрос: в 1959 году
Согласованность Ответа на вопрос, Вопроса, и Контекста: 1
['ответ', 'на', 'вопрос:', 'в', '1959', 'году\nсогласованность', 'ответа', 'на', 'вопрос,', 'вопроса,', 'и', 'контекста:', '1']
Predicted:  1
True:  0


Контекст: Пе́шка (Юникод: ♙♟) — самая слабая шахматная фигура и основная единица измерения шахматного материала: в пешечном эквиваленте измеряют «вес» фигур (лёгкая фигура примерно эквивалентна трём пешкам, ладья — пяти). По Ф. Филидору пешка — «душа шахматной партии», структура пешечного расположения определяет стратегический рисунок партии. В эндшпиле роль пешек значительно возрастает, особенно далеко продвинутых (близких к полю превращения). Буквенное обозначение 

KeyboardInterrupt: 

In [ ]:
result = df["answer"].tolist()

In [ ]:
results = list(map(lambda x: x[0], result))

In [ ]:

def get_sentence_transformers_embeddings(contexts,
                                         model_name="paraphrase-multilingual-MiniLM-L12-v2",
                                         batch_size=256):

    model = SentenceTransformer(model_name)

    embeddings = model.encode(contexts, batch_size=batch_size, show_progress_bar=True)
    print('embeddings shape', embeddings.shape)

    return embeddings

In [ ]:
embeddings = get_sentence_transformers_embeddings(results,
                                                  model_name="paraphrase-multilingual-MiniLM-L12-v2",
                                                  batch_size=32)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    embeddings, data['is_hallucination'], test_size=0.2, random_state=13, stratify=data['is_hallucination'])

In [ ]:
train_pd = pd.DataFrame(X_train.tolist(), columns=[str(i) for i in range(384)])
test_pd = pd.DataFrame(X_test.tolist(), columns=[str(i) for i in range(384)])

In [ ]:
scaling=StandardScaler()

scaling.fit(train_pd)
train_pd_sc=scaling.transform(train_pd)

# Set the n_components=3
principal=PCA(n_components=2)
#principal = TSNE(n_components=2, learning_rate='auto', init='random', perplexity=3)
principal.fit(train_pd_sc)
vect = principal.fit_transform(train_pd_sc)
vect

In [ ]:
px.scatter(
    y=vect[: , 0],
    x=vect[: , 1],
    color=y_train,
    title="Облако меток",
    template="plotly_dark",
)

In [ ]:
df.to_csv("/content/drive/MyDrive/Models/checkpoint_mistraldf.csv")

In [40]:
self_instruct_dir = 'rulm/self_instruct'
checkpoint = "../../model/checkpoint-20/"
merged_model_name = 'merged_test_model.pt'


In [ ]:
%cd {self_instruct_dir}

"ls" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


In [41]:
!python -m src.tools.convert_to_native {checkpoint} {merged_model_name} --device=cuda --enable_offloading

Saving state_dict...


Loading checkpoint shards: 100%|##########| 2/2 [00:07<00:00,  3.96s/it]

100%|##########| 291/291 [00:02<00:00, 100.75it/s]


In [56]:
model_dir = 'merged_test_model.pt'
checkpoint = "../model/checkpoint-20/"
output_model = "model-int8.gguf"

In [50]:
%cd llama.cpp

[WinError 2] Не удается найти указанный файл: 'llama.cpp'
E:\PyCharm Community Edition 2021.1.1\projects\LLM_LoRa\LLM_LoRa\llama.cpp


E:\PyCharm Community Edition 2021.1.1\projects\LLM_LoRa\venv\Lib\site-packages\IPython\core\magics\osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


In [60]:
!python examples/convert-legacy-llama.py {model_dir} --vocab-dir {checkpoint} --outfile {output_model} --outtype f16 --ctx 4096

INFO:convert:Loading model file merged_test_model.pt
INFO:convert:model parameters count : 6738415616 (7B)
INFO:convert:params = Params(n_vocab=32000, n_embd=4096, n_layer=32, n_ctx=4096, n_ff=11008, n_head=32, n_head_kv=32, n_experts=None, n_experts_used=None, f_norm_eps=1e-05, rope_scaling_type=None, f_rope_freq_base=None, f_rope_scale=None, n_ctx_orig=None, rope_finetuned=None, ftype=<GGMLFileType.MostlyF16: 1>, path_model=WindowsPath('.'))
INFO:convert:Loaded vocab file WindowsPath('../model/checkpoint-20/tokenizer.model'), type 'spm'
INFO:convert:Vocab info: <SentencePieceVocab with 32000 base tokens and 1 added tokens>
INFO:convert:Special vocab info: <SpecialVocab with 0 merges, special tokens unset, add special tokens unset>
INFO:convert:Writing model-int8.gguf, format 1
Traceback (most recent call last):
  File "E:\PyCharm Community Edition 2021.1.1\projects\LLM_LoRa\LLM_LoRa\llama.cpp\examples\convert-legacy-llama.py", line 1416, in <module>
    main()
  File "E:\PyCharm Comm

In [30]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("IlyaGusev/saiga2_7b_lora", use_fast=False)
tokenizer.save_pretrained('tokenizer')

tokenizer_config.json:   0%|          | 0.00/278 [00:00<?, ?B/s]

E:\PyCharm Community Edition 2021.1.1\projects\LLM_LoRa\venv\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user\.cache\huggingface\hub\models--IlyaGusev--saiga2_7b_lora. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/118 [00:00<?, ?B/s]

('tokenizer\\tokenizer_config.json',
 'tokenizer\\special_tokens_map.json',
 'tokenizer\\tokenizer.model',
 'tokenizer\\added_tokens.json')

In [ ]:
from llama_cpp import Llama
from tqdm import tqdm

SYSTEM_PROMPT = "Ты извлекаешь термины и определения из текста"
SYSTEM_TOKEN = 1788
USER_TOKEN = 1404
BOT_TOKEN = 9225
LINEBREAK_TOKEN = 13

top_k=40
top_p=0.5
temperature=0.01
repeat_penalty=1.1


ROLE_TOKENS = {
    "user": USER_TOKEN,
    "bot": BOT_TOKEN,
    "system": SYSTEM_TOKEN
}


def get_message_tokens(model, role, content):
    message_tokens = model.tokenize(content.encode("utf-8"))
    message_tokens.insert(1, ROLE_TOKENS[role])
    message_tokens.insert(2, LINEBREAK_TOKEN)
    message_tokens.append(model.token_eos())
    return message_tokens


def get_system_tokens(model):
    system_message = {
        "role": "system",
        "content": SYSTEM_PROMPT
    }
    return get_message_tokens(model, **system_message)

def chat_saiga(message, model):
    system_tokens = get_system_tokens(model)
    tokens = system_tokens
    # model.eval(tokens)
    
    message_tokens = get_message_tokens(model=model, role="user", content=message)
    role_tokens = [model.token_bos(), BOT_TOKEN, LINEBREAK_TOKEN]
    tokens += message_tokens + role_tokens
    # print(tokens)
    # detokenize = model.detokenize(tokens)
    # print(model.tokenize(full_prompt))
    generator = model.generate(
        tokens,
        top_k = top_k,
        top_p = top_p,
        temp = temperature,
        repeat_penalty = repeat_penalty,
        reset = True
    )
    # print(len([token for token in generator]))
    
    result_list = []
    
    for token in generator:
        token_str = model.detokenize([token]).decode("utf-8", errors="ignore")
        tokens.append(token)
        if token == model.token_eos():
            break
        print(token_str, end="", flush=True)
        result_list.append(token_str)
    return ''.join(result_list)

try:
    del model
except:
    pass

# model_path = '/kaggle/working/model-q4_0.gguf'
model_path = '/kaggle/working/model-q4_0.gguf'
n_ctx = 3096 #

model = Llama(
        model_path = model_path,
        n_ctx = n_ctx,
        n_gpu_layers=-1
)